# Imports

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/MURA-v1.1.zip

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For exampale, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
from skimage.color import rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.exposure import equalize_adapthist, adjust_gamma
import cv2
from tqdm import tqdm
import time
import gc
import shutil
from glob import glob

from sklearn.metrics import cohen_kappa_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Data loading and preprocessing

Load image paths.

In [0]:
train_image_paths = pd.read_csv('/content/MURA-v1.1/train_image_paths.csv')

Rename the column containing the path.

In [0]:
train_image_paths.columns = ['path']

Extract labels from folder names and correct all paths in our DataFrame.

In [0]:
labels = torch.tensor(train_image_paths['path'].str.contains('positive').astype('uint8').values).long()

train_image_paths['wrist'] = train_image_paths['path'].str.contains('WRIST').astype('uint8').values
train_image_paths['shoulder'] = train_image_paths['path'].str.contains('SHOULDER').astype('uint8').values
train_image_paths['hand'] = train_image_paths['path'].str.contains('HAND').astype('uint8').values
train_image_paths['elbow'] = train_image_paths['path'].str.contains('ELBOW').astype('uint8').values
train_image_paths['finger'] = train_image_paths['path'].str.contains('FINGER').astype('uint8').values
train_image_paths['forearm'] = train_image_paths['path'].str.contains('FOREARM').astype('uint8').values
train_image_paths['humerus'] = train_image_paths['path'].str.contains('HUMERUS').astype('uint8').values

train_image_paths['path'] = train_image_paths['path'].apply(lambda x: '/content/' + x)

In [0]:
train_image_paths

,path,wrist,shoulder,hand,elbow,finger,forearm,humerus
0,/content/MURA-v1.1/train/XR_SHOULDER/patient00...,0,1,0,0,0,0,0
1,/content/MURA-v1.1/train/XR_SHOULDER/patient00...,0,1,0,0,0,0,0
2,/content/MURA-v1.1/train/XR_SHOULDER/patient00...,0,1,0,0,0,0,0
3,/content/MURA-v1.1/train/XR_SHOULDER/patient00...,0,1,0,0,0,0,0
4,/content/MURA-v1.1/train/XR_SHOULDER/patient00...,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
36802,/content/MURA-v1.1/train/XR_HAND/patient11183/...,0,0,1,0,0,0,0
36803,/content/MURA-v1.1/train/XR_HAND/patient11183/...,0,0,1,0,0,0,0
36804,/content/MURA-v1.1/train/XR_HAND/patient11184/...,0,0,1,0,0,0,0
36805,/content/MURA-v1.1/train/XR_HAND/patient11184/...,0,0,1,0,0,0,0


In [0]:
len(train_image_paths), len(labels)

(36807, 36807)

In [0]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a.iloc[p], b[p]

train_image_paths, labels = unison_shuffled_copies(train_image_paths, labels)

In [0]:
train_image_paths.head()

,path,wrist,shoulder,hand,elbow,finger,forearm,humerus
11999,/content/MURA-v1.1/train/XR_FINGER/patient0391...,0,0,0,0,1,0,0
8592,/content/MURA-v1.1/train/XR_HUMERUS/patient025...,0,0,0,0,0,0,1
36374,/content/MURA-v1.1/train/XR_HAND/patient11065/...,0,0,1,0,0,0,0
16452,/content/MURA-v1.1/train/XR_ELBOW/patient05357...,0,0,0,1,0,0,0
21488,/content/MURA-v1.1/train/XR_WRIST/patient06813...,1,0,0,0,0,0,0


In [0]:
OUT_DIM = (320, 320)

Split dataset to chunks of size ±1000.

In [0]:
label_chunks = np.array_split(labels, 70)
image_path_chunks = np.array_split(train_image_paths['path'].values, 70)

data_chunks = [np.array(image_path_chunks), label_chunks]

In [0]:
print('Amount of true labels: {}, false labels: {}'.format(len(labels[labels == 1]), len(labels[labels == 0])))

Amount of true labels: 14872, false labels: 21935


In [0]:
import albumentations as alb

aug_amount = 3

aug = alb.Compose([
    alb.ShiftScaleRotate(rotate_limit=20, scale_limit=(-0.02, 0.02), p=1, border_mode=cv2.BORDER_CONSTANT),
#     alb.RandomContrast(p=0.5)
])

In [0]:
def preprocess(image_paths, do_aug=True):
    if do_aug:
        images = torch.zeros(len(image_paths) * (aug_amount + 1), 3, OUT_DIM[0], OUT_DIM[1])
    
        for i, image_path in enumerate(tqdm(image_paths, position=0, leave=True)):
            image = gray2rgb(resize(imread(image_path), OUT_DIM, mode='constant'))
            for j in range(aug_amount):
                aug_image = aug(image=image)['image']
                aug_image = np.swapaxes(aug_image, -1, 0)
                images[i, :, :, :] = torch.tensor(aug_image)
                i += 1
            image = np.swapaxes(image, -1, 0)
            images[i, :, :, :] = torch.tensor(image)
    else:
        images = torch.zeros(len(image_paths), 3, OUT_DIM[0], OUT_DIM[1])
    
        for i, image_path in enumerate(tqdm(image_paths, position=0, leave=True)):
            image = gray2rgb(resize(imread(image_path), OUT_DIM, mode='constant'))
            image = np.swapaxes(image, -1, 0)
            images[i, :, :, :] = torch.tensor(image)
        
    return images

# Model

I am using a pretrained DenseNet model. It has already learnt a set of features, so we can save up time training all the layers ourselves.

In [0]:
!export CUDA_LAUNCH_BLOCKING=1

In [16]:
from torchvision import models

densenet169 = models.densenet169(pretrained=True)
densenet169.classifier = nn.Linear(in_features=1664, out_features=2, bias=True)
densenet169.cuda()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

As we have a simple binary classification, we can use weighted Cross entropy loss function. The optimizer is Adam.


In [0]:
A = len(labels[labels == 1])
N = len(labels[labels == 0])

W1 = N / (A + N)
W0 = A / (A + N)

In [0]:
learning_rate = 0.0001

criterion = nn.CrossEntropyLoss(weight=torch.tensor([W0, W1]).cuda())
optimizer = torch.optim.Adam(densenet169.parameters(), lr=learning_rate)

Training part was borrowed from [here (clickable)](https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py)

In [0]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        images = normalize(images, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        target = target.long().cuda(async=True)
        input_var = Variable(images).cuda()
        target_var = Variable(target)

        # compute output
        output = model(input_var)
        pred = np.argmax(output.data.cpu(), axis=1)

        loss = criterion(output, target_var)

        # measure accuracy and record loss
        f1 = f1_score(pred, target_var.cpu())

        losses.update(loss.data, images.size(0))
        top1.update(f1, images.size(0))        

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 10 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Kappa {f1_score:.4f}\t'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, f1_score=f1))


def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (images, target) in enumerate(val_loader):
        target = target.cuda(async=True)
        images = normalize(images, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        target = target.long().cuda(async=True)
        with torch.no_grad():
            input_var = torch.autograd.Variable(images).cuda()
            target_var = torch.autograd.Variable(target).cuda()

            # compute output
            output = model(input_var)
            pred = np.argmax(output.data.cpu(), axis=1)

            loss = criterion(output, target_var)

            # measure accuracy and record loss
            f1 = f1_score(pred, target_var.cpu())

            losses.update(loss.data, images.size(0))
            top1.update(kappa_score, images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        if i % 10 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Kappa {f1_score:.4f}\t'.format(
                   i, len(val_loader), batch_time=batch_time,
                   loss=losses, f1_score=f1))

    return top1.avg


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = learning_rate * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def normalize(tensor, mean, std):
    for i in range(3):
        if len(tensor.shape) == 4:
            tensor[:, i, :, :] = (tensor[:, i, :, :] - mean[i]) / std[i]
        else:
            tensor[i, :, :] = (tensor[i, :, :] - mean[i]) / std[i]
        
    return tensor

In [0]:
from sklearn.model_selection import train_test_split

batch_size = 8
best_prec1 = 0.0

for image_paths, labels in zip(data_chunks[0][:5], data_chunks[1][:5]):
    images = preprocess(image_paths)
    labels = np.repeat(labels, aug_amount + 1) # due to the augmentation

    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.25, random_state=17)
    
    train_ds = TensorDataset(X_train, y_train)
    val_ds = TensorDataset(X_val, y_val)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=4)
    for epoch in range(5):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        train(train_loader, densenet169, criterion, optimizer, epoch)

        # evaluate on validation set
        prec1 = validate(val_loader, densenet169, criterion)

        # remember best prec1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': 'densenet169',
            'state_dict': densenet169.state_dict(),
            'best_prec1': best_prec1,
        }, is_best)

100%|██████████| 526/526 [00:14<00:00, 35.31it/s]


748
1356
Epoch: [0][0/198]	Time 1.536 (1.536)	Data 1.266 (1.266)	Loss 0.7955 (0.7955)	Kappa 0.0000	
Epoch: [0][10/198]	Time 0.221 (0.370)	Data 0.003 (0.118)	Loss 0.6864 (0.7094)	Kappa 0.0000	


KeyboardInterrupt: ignored

In [0]:
pretrained_dict = torch.load('/content/model_best.pth.tar')['state_dict']
densenet169_dict = densenet169.state_dict()

# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in densenet169_dict}
# 2. overwrite entries in the existing state dict
densenet169_dict.update(pretrained_dict)
# 3. load the new state dict
densenet169.load_state_dict(pretrained_dict)

<All keys matched successfully>

In [0]:
test_folders = glob('/content/MURA-v1.1/valid/*/*/*')
test_folders[:3]

['/content/MURA-v1.1/valid/XR_WRIST/patient11226/study1_positive',
 '/content/MURA-v1.1/valid/XR_WRIST/patient11347/study2_negative',
 '/content/MURA-v1.1/valid/XR_WRIST/patient11347/study1_negative']

In [0]:
class PatientCase:
    def __init__(self, image_paths, label):
        self.image_paths = image_paths
        self.label = label
        
    def get_image_paths():
        return self.image_paths

In [0]:
all_cases = []

for test_folder in test_folders:
    files = glob(test_folder + '/*')
    label = 1  if 'positive' in test_folder.split('_') else 0
    all_cases.append(PatientCase(files, label))

In [25]:
total_pred = []
labels = []

for case in all_cases:
    with torch.no_grad():
        test_images = preprocess(case.image_paths, do_aug=False)
        test_images = normalize(test_images, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        labels.append(case.label)
        pred = densenet169(Variable(test_images).cuda())
        total_pred.append(pred.cpu().numpy())

100%|██████████| 3/3 [00:00<00:00, 78.32it/s]


In [26]:
total_pred[90:100]

[array([[ 0.01468022,  0.02140172],
        [-0.07101468,  0.11038935],
        [-0.06822452,  0.12852825]], dtype=float32),
 array([[-0.00805898,  0.07730801],
        [-0.08754686,  0.09517364]], dtype=float32),
 array([[ 0.01875428,  0.03255735],
        [-0.0533618 ,  0.08517505],
        [-0.09526484,  0.13394812]], dtype=float32),
 array([[-0.03904623,  0.09296435]], dtype=float32),
 array([[-0.06032981,  0.10745808],
        [-0.02167301,  0.09290321],
        [-0.03365557,  0.04225169]], dtype=float32),
 array([[-0.0547958 ,  0.08379893]], dtype=float32),
 array([[-0.04470194,  0.11249726],
        [-0.04817079,  0.107112  ],
        [-0.03164651,  0.05817854]], dtype=float32),
 array([[-0.05221235,  0.12757571],
        [-0.0383327 ,  0.07056088],
        [-0.04878949,  0.06289744]], dtype=float32),
 array([[-0.08710052,  0.09216995],
        [-0.00874134,  0.07867004]], dtype=float32),
 array([[-0.02413774,  0.06731763],
        [-0.04564085,  0.07395231],
        [-0.0670903

In [0]:
res = []

for pred in total_pred:
  res.append(np.argmax(np.sum(pred, axis=0) / len(pred)))

# res

In [32]:
f1_score = f1_score(res, labels)

TypeError: ignored

In [30]:
f1_score

0.6194588370754174